## Modelo de recomendación de juegos
En esta jupyter notebook haremos la experimentación para encontrar dos modelos de recomendación, que generen una lista de 5 juegos ya sea ingresando el nombre de un juego o el id de un usuario.

En el primer caso, el modelo tiene una relación ítem-ítem, esto significa que tomaremos un juego y recomendaremos similares en base a qué tan similar es ese juego con el resto de los otros. En el segundo caso, el modelo aplicar un filtro usuario-juego, es decir, tomaremos un usuario, encontraremos usuarios similares y recomendaremos ítems que a esos usuarios similares les gustaron.

Para generar estos modelos se adoptaron algoritmos basados en la memoria, los que abordan el problema del filtrado colaborativo utilizando toda la base de datos, tratando de encontrar usuarios similares al usuario activo (es decir, los usuarios para los que se les quiere recomendar) y utilizando sus preferencias para predecir las valoraciones del usuario activo.

In [1]:
import pandas as pd
import numpy as np
import operator
import pyarrow as pa
import pyarrow.parquet as pq
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df_recomendacion = pd.read_csv('df_recomendacion.csv')
df_recomendacion

,user_id,item_name,puntaje
0,--000--,PlanetSide 2,5
1,112asdasfasdasd,PlanetSide 2,3
2,1234567890192837465,PlanetSide 2,3
3,2828838282,PlanetSide 2,3
4,2sd31,PlanetSide 2,3
...,...,...,...
44196,yougotblehed,A.R.E.S. Extinction Agenda EX,2
44197,yougoyu,Wolcen: Lords of Mayhem,3
44198,zayyntt,Cuties,3
44199,zayyntt,Neon Space ULTRA,3


El primer paso es crear un dataframe que contiene los "user_id" como idices, los juegos "item_name" como columnas y como valores los "puntaje".

In [4]:
pivot = df_recomendacion.pivot_table(index=["user_id"], columns=["item_name"], values="puntaje")
pivot

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,140,16bit Trader,...,inMomentum,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzuga2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A continuación, normalizaremos los valores del dataFrame pivot, restaremos la media de las calificaciones de un usuario y luego, dividiremos por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario de manera que estén centradas en cero y escaladas en función de su variabilidad. A los usuarios que solo han dado una calificación o han calificado todos los juegos de la misma manera serán eliminados durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

In [5]:
# Normalización del dataframe 'pivot'
pivot_norm = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Borramos las columnas que contienen solo cero o no tienen puntaje, rellenamos los vacios con 0 y hacemos la transpuesta
pivot_norm.fillna(0, inplace=True)
pivot_norm = pivot_norm.T
pivot_norm = pivot_norm.loc[:, (pivot_norm != 0).any(axis=0)]
pivot_norm

user_id,--000--,-Beave-,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,03092002,04061993,...,zombi_anon,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zsharoarkbr,zuzuga2003,zvanik,zwanzigdrei,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos de esta matriz normalizada, los convertiremos a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros. La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

In [6]:
pivot_sparse = sp.sparse.csr_matrix(pivot_norm.values)
pivot_sparse

<2809x6963 sparse matrix of type '<class 'numpy.float64'>'
	with 25036 stored elements in Compressed Sparse Row format>

Ahora, crearemos dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

La similitud del coseno es una medida comúnmente utilizada para evaluar la similitud entre dos vectores en un espacio multidimensional. En el contexto de sistemas de recomendación y análisis de datos, la similitud del coseno se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [7]:
item_similarity = cosine_similarity(pivot_sparse)
user_similarity = cosine_similarity(pivot_sparse.T)

Para estructurar y organizar los resultados de manera más accesible y comprensible, insertaremos las matrices anteriores en un Dataframe.

In [8]:
#item similarity dataframe
df_item_sim = pd.DataFrame(item_similarity, index = pivot_norm.index, columns = pivot_norm.index)
#user similarity dataframe
df_user_sim = pd.DataFrame(user_similarity, index = pivot_norm.columns, columns = pivot_norm.columns)


## Recomendación según un juego
Ahora, conociendo la relación entre los distintos juegos, se puede proponer una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno. Esta función toma un nombre de un juego como entrada, luego ordena la columna correspondiente a ese juego en la matriz de similitud entre elementos (item_sim_df) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.

In [9]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función presenta una lista de juegos 5 similares al dado.

    '''
    count = 1
    print("Similar games to {} include:\n".format(game))
    for item in df_item_sim.sort_values(by = game, ascending = False).index[1:6]:
        print("No. {}: {}".format(count, item))
        count +=1 

In [10]:
top_game("resident evil 4 / biohazard 4")

Similar games to resident evil 4 / biohazard 4 include:

No. 1: Assassin's Creed Liberation
No. 2: Resident Evil 6 / Biohazard 6
No. 3: NARUTO SHIPPUDEN: Ultimate Ninja STORM 4
No. 4: Shadow Warrior
No. 5: FINAL FANTASY XIII


In [11]:
top_game("100% Orange Juice")

Similar games to 100% Orange Juice include:

No. 1: RUNNING WITH RIFLES
No. 2: Planet Explorers
No. 3: Eryi's Action
No. 4: Talisman: Digital Edition
No. 5: Dark Souls: Prepare to Die Edition


### Recomendación según un usuario
La siguiente función tiene la finalidad de mostrar una lista de usuarios más similares a un usuario dado, junto con sus valores de similitud. Primero, ordena la matriz de similitud entre usuarios (df_user_sim) en orden descendente según la similitud con el usuario dado ("user_id"), luego toma los 5 usuarios más similares (excluyendo el propio usuario) y almacena sus nombres y valores de similitud en las listas sim_users y sim_values. Finalmente, combina los nombres de usuario y los valores de similitud en una lista de tuplas utilizando la función zip y los imprime.

Esto es útil en sistemas de recomendación para mostrar a un usuario los usuarios más similares en función de sus calificaciones pasadas, lo que puede ayudar en la generación de recomendaciones personalizadas.

In [13]:
def top_users(user):
    '''
    Muestra una lista de los usuarios más similares a un usuario dado y sus valores de similitud.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean encontrar usuarios similares.

    Returns:
        None: Esta función imprime la lista de usuarios similares y sus valores de similitud en la consola.

    '''
    # Verificamos si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in pivot_norm.columns:
        return("No data available on user {}".format(user))
    
    print("Most Similar Users:\n")
    # Ordenamos los usuarios por similitud descendente y toma los 5 usuarios más similares (excluyendo el propio "user")
    sim_values = df_user_sim.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:6]
    sim_users = df_user_sim.sort_values(by=user, ascending=False).index[1:11]
    # Combinamos los nombres de usuario y los valores de similitud en una lista de tuplas
    zipped = zip(sim_users, sim_values,)
    
    # Itera a través de las tuplas y muestra los usuarios similares y sus valores de similitud
    for user, sim in zipped:
        print("User #{0}, Similarity value: {1:.2f}".format(user, sim)) 


In [14]:
top_users("zuzuga2003")

Most Similar Users:

User #Reaper_of_the_winds, Similarity value: 0.91
User #ThaFiddy, Similarity value: 0.87
User #ThereInTheTrees, Similarity value: 0.83
User #triote50, Similarity value: 0.82
User #Jacler, Similarity value: 0.73


En esta función, generamos una lista de 5 juegos recomendados para un usuario en función de las calificaciones de usuarios similares. Los juegos que son más frecuentemente recomendados por usuarios similares se consideran como las principales recomendaciones para ese usuario.

In [15]:
def similar_user_recs(user):
    '''
    Genera una lista de los juegos más recomendados para un usuario, basándose en las calificaciones de usuarios similares.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean generar recomendaciones.

    Returns:
        list: Una lista de los juegos más recomendados para el usuario basado en la calificación de usuarios similares.

    '''
    # Verificamos si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in pivot_norm.columns:
        return("No data available on user {}".format(user))
    
    # Obtenemos los usuarios más similares al usuario dado
    sim_users = df_user_sim.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista "best"
    for i in sim_users:
        max_score = pivot_norm.loc[:, i].max()
        best.append(pivot_norm[pivot_norm.loc[:, i]==max_score].index.tolist())
    
    # Cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Ordenamos los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Devuelve los 5 juegos más recomendados
    return sorted_list[:5]  

In [16]:
similar_user_recs("zuzuga2003")

[('Counter-Strike: Global Offensive', 8),
 ('Unturned', 1),
 ('Blacklight: Retribution', 1),
 ('Chivalry: Medieval Warfare', 1),
 ('STAR WARS™ Jedi Knight: Jedi Academy™', 1)]

## Carga de las matrices para recomendar en la API
Para poder utilizar las funciones top_users y similar_user_recs es necesario consumir las matrices piv_norm y user_sim_df. Por ello, se guardan, en este caso, en formato parquet los que permite una compresión y codificación eficiente.

In [17]:
pq.write_table(pa.Table.from_pandas(pivot_norm), 'pivot_norm.parquet')
pq.write_table(pa.Table.from_pandas(df_user_sim), 'df_user_sim.parquet')
pq.write_table(pa.Table.from_pandas(df_item_sim), 'df_item_sim.parquet')
print('Se guardaron correctamente')

Se guardaron correctamente
